# Load Depenency

In [60]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)


/Users/ryoung/opt/anaconda3/envs/da622/bin/python
3.10.4 (main, Mar 31 2022, 03:38:35) [Clang 12.0.0 ]
sys.version_info(major=3, minor=10, micro=4, releaselevel='final', serial=0)


In [61]:
from os import system
import pandas as pd
import requests
import datetime as dt
# from pytrials.client import ClinicalTrials
import json
import ipywidgets as widgets
import numpy as np
np.random.seed(10031975)
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
# from openpyxl import Workbook
import openpyxl

does some error handling

In [62]:
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [63]:
from pytrials.utils import json_handler, csv_handler


class ClinicalTrials:
    """ClinicalTrials API client

    Provides functions to easily access the ClinicalTrials.gov API
    (https://clinicaltrials.gov/api/)
    in Python.

    Attributes:
        study_fields: List of all study fields you can use in your query.
        api_info: Tuple containing the API version number and the last
        time the database was updated.
    """

    _BASE_URL = "https://clinicaltrials.gov/api/"
    _INFO = "info/"
    _QUERY = "query/"
    _JSON = "fmt=json"
    _CSV = "fmt=csv"

    def __init__(self):
        self.api_info = self.__api_info()

    @property
    def study_fields(self):
        fields_list = json_handler(
            f"{self._BASE_URL}{self._INFO}study_fields_list?{self._JSON}"
        )
        return fields_list["StudyFields"]["Fields"]

    def __api_info(self):
        """Returns information about the API"""
        last_updated = json_handler(
            f"{self._BASE_URL}{self._INFO}data_vrs?{self._JSON}"
        )["DataVrs"]
        api_version = json_handler(f"{self._BASE_URL}{self._INFO}api_vrs?{self._JSON}")[
            "APIVrs"
        ]

        return api_version, last_updated

    def get_full_studies(self, search_expr, max_studies=50):
        """Returns all content for a maximum of 100 study records.

        Retrieves information from the full studies endpoint, which gets all study fields.
        This endpoint can only output JSON (Or not-supported XML) format and does not allow
        requests for more than 100 studies at once.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.

        Returns:
            dict: Object containing the information queried with the search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 100
        """
        if max_studies > 100 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 100")

        req = f"full_studies?expr={search_expr}&max_rnk={max_studies}&{self._JSON}"

        full_studies = json_handler(f"{self._BASE_URL}{self._QUERY}{req}")

        return full_studies

    def get_study_fields(self, search_expr, fields, max_studies=50, min_rnk=1,fmt="csv"):
        """Returns study content for specified fields

        Retrieves information from the study fields endpoint, which acquires specified information
        from a large (max 1000) studies. To see a list of all possible fields, check the class'
        study_fields attribute.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            fields (list(str)): A list containing the desired information fields.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.
            fmt (str): A string indicating the output format, csv or json. Defaults to csv.

        Returns:
            Either a dict, if fmt='json', or a list of records (e.g. a list of lists), if fmt='csv.
            Both containing the maximum number of study fields queried using the specified search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 1000
            ValueError: One of the fields is not valid! Check the study_fields attribute
                for a list of valid ones.
            ValueError: Format argument has to be either 'csv' or 'json'
        """
        if max_studies > 1000 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 1000")
        elif not set(fields).issubset(self.study_fields):
            raise ValueError(
                "One of the fields is not valid! Check the study_fields attribute for a list of valid ones."
            )
        else:
            concat_fields = ",".join(fields)
            # req = f"study_fields?expr={search_expr}&max_rnk={max_studies}&fields={concat_fields}"
            req = f"study_fields?expr={search_expr}&min_rnk={min_rnk}&max_rnk={max_studies+min_rnk-1}&fields={concat_fields}"
            if fmt == "csv":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._CSV}"
                return csv_handler(url)

            elif fmt == "json":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
                return json_handler(url)

            else:
                raise ValueError("Format argument has to be either 'csv' or 'json'")

    def get_study_count(self, search_expr):
        """Returns study count for specified search expression

        Retrieves the count of studies matching the text entered in search_expr.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.

        Returns:
            An integer

        Raises:
            ValueError: The search expression cannot be blank.
        """
        if not set(search_expr):
            raise ValueError("The search expression cannot be blank.")
        else:
            req = f"study_fields?expr={search_expr}&max_rnk=1&fields=NCTId"
            url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
            returned_data = json_handler(url)
            study_count = returned_data["StudyFieldsResponse"]["NStudiesFound"]
            return study_count

    def __repr__(self):
        return f"ClinicalTrials.gov client v{self.api_info[0]}, database last updated {self.api_info[1]}"

# Load Data

https://clinicaltrials.gov/api/gui/ref/crosswalks

Disease 2 search for

This is for dymanic input of diease searching

In [64]:
# disease=input()
# print(disease)

In [65]:
disease = 'tdcs'

#disease = 'depression'

# disease = 'diabetes'

# disease = 'tacs'



hit mesh some day

Pull ONE

#Explore Data

In [66]:
#change this out to NAME
# from turtle import up


# need to clean this up

ct = ClinicalTrials()
# infodf= (ct.get_full_studies(search_expr=disease, max_studies=5 ))
# number = print(infodf['FullStudiesResponse']['NStudiesFound'])
count = ct.get_study_count(search_expr=disease)
print(count)



1478


In [67]:
# count = 3000
start = 0
end = count
iter = 800 #number of studies to get at a time ( limited to 1000 )





temp = pd.DataFrame()

temp2 = pd.DataFrame()
new_df = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        # you can pick from here
        # https://www.clinicaltrials.gov/api/info/study_fields_list
        fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType", "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase","StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BriefSummary","IsFDARegulatedDrug","IsFDARegulatedDevice"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    iter2 = iter + (i-1)
    print(str(i) + "-" + str(iter2) + " of " + str(count))
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df = pd.concat([new_df, temp2])
 


1-800 of 1478
801-1600 of 1478


In [68]:
new_df.columns = ["Index","NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BriefSummary","IsFDARegulatedDrug","IsFDARegulatedDevice"]

In [69]:
# new_df
new_df.shape

(1478, 19)

In [70]:
# new_df.to_excel(disease+'-a'+'.xlsx',  engine='openpyxl', index = False)

In [71]:
new_df.head

<bound method NDFrame.head of     Index        NCTId                                          Condition  \
0       1  NCT03288363                                  Alzheimer Disease   
1       2  NCT02161302                         Chronic Pain|Endometriosis   
2       3  NCT04759092                                  Alzheimer Disease   
3       4  NCT03879044                          Shift-Work Sleep Disorder   
4       5  NCT02665988                 Chronic Pain|Psychiatric Disorders   
..    ...          ...                                                ...   
673  1474  NCT04545294  Schizophrenia|Schizoaffective Disorder|Negativ...   
674  1475  NCT05429736   Incomplete Spinal Cord Injury|Spasticity, Muscle   
675  1476  NCT04038788  Schizophrenia|Schizoaffective Disorder|Psychot...   
676  1477  NCT00801489  Acute Myeloid Leukemia With Inv(16)(p13.1q22);...   
677  1478  NCT00416598  Acute Myeloid Leukemia|Acute Myeloid Leukemia ...   

                                         Offi

Ok - so the reason for 2 pulls is because for some reason if I request more than ~20 columns the API has an issue


In [72]:
# count = 3000
start = 0
end = count
iter = 400 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df2 = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId", "ConditionMeshTerm","PrimaryOutcomeDescription", "SecondaryOutcomeDescription", "EnrollmentCount","EnrollmentType","BaselineCategoryTitle","BaselinePopulationDescription","BaselineTypeUnitsAnalyzed",
"OtherOutcomeDescription","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity",  "LocationCountry" , "BaselineMeasureTitle", "BaselineMeasureUnitOfMeasure", "BaselineMeasurementValue"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    iter2 = iter + (i-1)
    print(str(i) + "-" + str(iter2) + " of " + str(count))
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df2 = pd.concat([new_df2, temp2])
 


1-400 of 1478
401-800 of 1478
801-1200 of 1478
1201-1600 of 1478


In [73]:
new_df2.columns = ["index", "NCTId", "ConditionMeshTerm","PrimaryOutcomeDescription", "SecondaryOutcomeDescription", "EnrollmentCount","EnrollmentType","BaselineCategoryTitle","BaselinePopulationDescription","BaselineTypeUnitsAnalyzed",
"OtherOutcomeDescription","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationCountry" , "BaselineMeasureTitle", "BaselineMeasureUnitOfMeasure", "BaselineMeasurementValue"]

In [74]:
new_df2.shape

(1478, 20)

##cleaning

used to save both dataframes - but it is kinda of a pain to have to both on the file system, so just going to save after merge

In [75]:
# new_df.to_excel(disease+'-b'+'.xlsx',  engine='openpyxl', index = False)

In [76]:
# print(type(df1))

In [77]:
result = pd.merge(new_df, new_df2,  on='NCTId', how='left')

In [78]:
result.shape

(1478, 38)

IMPORTANT - this just drops the second index column - monitor or watch for mismatch or if you change the number of elements that are pulled from the first api

In [79]:
# result.head()
i=-1
for col in result.columns:
   i=i+1
   print(i, col)

0 Index
1 NCTId
2 Condition
3 OfficialTitle
4 BriefTitle
5 Acronym
6 StudyType
7 InterventionType
8 InterventionName
9 InterventionOtherName
10 InterventionDescription
11 Phase
12 StudyFirstSubmitDate
13 LastUpdateSubmitDate
14 CompletionDate
15 OverallStatus
16 BriefSummary
17 IsFDARegulatedDrug
18 IsFDARegulatedDevice
19 index
20 ConditionMeshTerm
21 PrimaryOutcomeDescription
22 SecondaryOutcomeDescription
23 EnrollmentCount
24 EnrollmentType
25 BaselineCategoryTitle
26 BaselinePopulationDescription
27 BaselineTypeUnitsAnalyzed
28 OtherOutcomeDescription
29 EligibilityCriteria
30 StudyPopulation
31 HealthyVolunteers
32 ReferencePMID
33 LocationCity
34 LocationCountry
35 BaselineMeasureTitle
36 BaselineMeasureUnitOfMeasure
37 BaselineMeasurementValue


In [80]:
result = result.drop(result.columns[[19]], axis=1)

In [81]:
# result.shape

Save file

In [82]:
# writer = pd.ExcelWriter('test.xlsx')
# # write dataframe to excel
# result.to_excel(writer)
# # save the excel
# writer.save()
# print('DataFrame is written successfully to Excel File.')

In [83]:
result

,Index,NCTId,Condition,OfficialTitle,BriefTitle,Acronym,StudyType,InterventionType,InterventionName,InterventionOtherName,...,OtherOutcomeDescription,EligibilityCriteria,StudyPopulation,HealthyVolunteers,ReferencePMID,LocationCity,LocationCountry,BaselineMeasureTitle,BaselineMeasureUnitOfMeasure,BaselineMeasurementValue
0,1,NCT03288363,Alzheimer Disease,Effects at 12 Weeks of Transcranial Direct Cur...,Transcranial Direct Current Stimulation and Ea...,tDCS-AD,Interventional,Device,tDCS,transcranial direct current stimulation,...,,Inclusion Criteria:||A man or woman with proba...,,No,,Limoges,France,,,
1,2,NCT02161302,Chronic Pain|Endometriosis,THE EFFECT OF TRANSCRANIAL DIRECT CURRENT STIM...,THE EFFECT OF tDCS IN THE TREATMENT OF CHRONIC...,tDCS,Interventional,Device|Device,transcranial direct current stimulation (tDCS)...,tDCS|Sham tDCS,...,Depression symptoms will be assessed by using ...,"Inclusion Criteria:||age > 18 years old, non-m...",,No,21170277|20945016|18312584|14677403|20633386|2...,Porto Alegre,Brazil,,,
2,3,NCT04759092,Alzheimer Disease,A Four Month Home Based tDCS Study on Patients...,A Four Month Home Based tDCS Study on Patients...,tDCS,Interventional,Device,transcranial direct current stimulation (tDCS),tDCS,...,,Inclusion Criteria:||Age 60-85.|Fulfil diagnos...,,No,35852094,Tromsø,Norway,,,
3,4,NCT03879044,Shift-Work Sleep Disorder,Transcranial Direct Current Stimulation Therap...,Transcranial Direct Current Stimulation Therap...,tDCS-SWORD,Interventional,Device,Transcranial Direct Current Stimulation,tDCS,...,,Inclusion Criteria:||age 18-65 years|works 5 o...,,No,,Columbus,United States,"Age, Continuous|Sex: Female, Male|Ethnicity (N...",years|Participants|Participants|Participants|P...,31.33|24|2|1|24|1|0|2|0|4|18|2|0|26
4,5,NCT02665988,Chronic Pain|Psychiatric Disorders,"A Single-Blind, Randomized Control Trial of Ad...",Adjunctive Transcranial Direct Current Stimula...,tDCS,Interventional,Device|Device,tDCS|Sham tDCS,transcranial Direct Current Stimulation|Sham t...,...,,Inclusion Criteria:||Males and females greater...,,No,,Houston,United States,"Age, Categorical|Sex: Female, Male|Race (NIH/O...",Participants|Participants|Participants|partici...,0|0|0|2|2|4|0|0|0|2|2|4|0|0|0|0|0|0|0|0|0|0|0|...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1473,1474,NCT04545294,Schizophrenia|Schizoaffective Disorder|Negativ...,Online Left-hemispheric Frontoparietal Theta I...,Theta tACS During Working Memory Training: A P...,,Interventional,Device,Theta tACS,,...,,Inclusion Criteria:||Eligible participants age...,,No,27023427|25477826|27930934|23993864|25528757|2...,Taipei,Taiwan,,,
1474,1475,NCT05429736,"Incomplete Spinal Cord Injury|Spasticity, Muscle",SCIMS Site Specific Project: Activating Spinal...,"Activating Spinal Circuits to Improve Walking,...",,Interventional,Behavioral|Device|Behavioral,Motor Skill Training|Transcutaneous Spinal Sti...,TSS|tSCS|CAX,...,,Inclusion Criteria:||Have a spinal cord injury...,,No,,Atlanta,United States,,,
1475,1476,NCT04038788,Schizophrenia|Schizoaffective Disorder|Psychot...,The Effects of High-Definition High-Frequency ...,Add-on HD Hf-tRNS Over Bilateral DLPFC for Tre...,,Interventional,Device,HD hf-tRNS,,...,,Inclusion Criteria:||Eligible participants age...,,No,33096356,Taipei,Taiwan,,,
1476,1477,NCT00801489,Acute Myeloid Leukemia With Inv(16)(p13.1q22);...,"A Phase 2 Study of Fludarabine, Cytarabine, Fi...","Fludarabine Phosphate, Cytarabine, Filgrastim-...",,Interventional,Drug|Drug|Biological|Drug|Drug|Drug|Other,Cytarabine|Decitabine|Filgrastim-sndz|Fludarab...,.beta.-Cytosine arabinoside|1-.beta.-D-Arabino...,...,,Inclusion Criteria:||Patients must have untrea...,,No,,Houston,United States,,,


In [84]:
# This simple method provide erros when open in excel because of the data from the API. I have to clean it up and then I will post the code.

# result.to_excel(disease+'_data_merged.xlsx',  engine='openpyxl', index = False)



In [85]:
# from lib2to3.pytree import _Results

import xlsxwriter

options = {}
options['strings_to_formulas'] = False
options['strings_to_urls'] = False
writer = pd.ExcelWriter(('1-'+disease+'_starting_Data.xlsx'),engine='xlsxwriter',options=options)
result.to_excel(writer, 'Sheet_A', index=False)
writer.save()

/var/folders/7x/knvg10qn19740t2kqn0qwy_40000gn/T/ipykernel_8850/2487760248.py:8: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter(('1-'+disease+'_starting_Data.xlsx'),engine='xlsxwriter',options=options)


In [86]:
# result.to_excel(disease+'_data_merged.xlsx',sheet_name='Sheet1',  engine='openpyxl', index = False)

In [87]:
#backup because csv works with strange characters

result.to_csv('1-'+disease+'_starting_data_bk.csv', index = False)

In [88]:
# result.to_hdf('data_raw.h5', key='df', mode='w')  